In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin 

def download_one_episode(title, no, url):
    print(f"\n--- 웹툰 다운로드 시작 ---")
    print(f"웹툰 제목: {title}, 회차 번호: {no}, URL: {url}")

    req_header = {
        'Referer': url, 
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    try:
        res = requests.get(url, headers=req_header)
        res.raise_for_status() # HTTP 오류가 발생하면 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"오류: 웹툰 페이지 요청 실패 - {e}")
        return

    # 페이지가 성공적으로 로드되었는지 확인
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        # 웹툰 이미지를 포함하는 뷰어 영역을 찾음
        # 네이버 웹툰은 일반적으로 'div.wt_viewer' 안에 이미지들이 있음
        viewer_div = soup.select_one('div.wt_viewer')

        if not viewer_div:
            print("오류: 웹툰 이미지를 포함하는 뷰어 영역(div.wt_viewer)을 찾을 수 없습니다.")
            print("페이지 HTML 소스를 확인하여 정확한 선택자를 찾아주세요.")
            return

        imgurl_list = []
        for img_tag in viewer_div.find_all('img'):
            src = img_tag.get('src')
            data_src = img_tag.get('data-src')

            if src and src.startswith('https://image-comic.pstatic.net/webtoon/'):
                imgurl_list.append(src)
            elif data_src and data_src.startswith('https://image-comic.pstatic.net/webtoon/'):
                imgurl_list.append(data_src)
            
        if not imgurl_list:
            print(f"오류: '{title}' {no}회차에서 다운로드할 이미지를 찾을 수 없습니다.")
            print("이미지 선택자를 확인하거나, 페이지 구조가 변경되었을 수 있습니다.")
            return

        print(f"총 {len(imgurl_list)}개의 이미지를 찾았습니다.")

        # 이미지를 저장할 디렉토리 생성
        # 예: img/일렉시드/341
        safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '.', '_', '-')).strip()
        dir_name = os.path.join('img', safe_title, str(no))
        
        print(f"이미지 저장 디렉토리: {dir_name}")

        if not os.path.isdir(dir_name):
            try:
                os.makedirs(dir_name)
                print(f"디렉토리 '{dir_name}' 생성 완료.")
            except OSError as e:
                print(f"오류: 디렉토리 '{dir_name}' 생성 실패 - {e}")
                return

        # 각 이미지 다운로드 및 저장
        for idx, img_url in enumerate(imgurl_list, 1):
            try:
                # 이미지 URL이 상대 경로일 경우를 대비하여 절대 경로로 변환
                full_img_url = urljoin(url, img_url)

                img_res = requests.get(full_img_url, headers=req_header, stream=True) 
                img_res.raise_for_status()

                file_name = os.path.basename(full_img_url)
                file_path = os.path.join(dir_name, file_name)

                with open(file_path, 'wb') as file:
                    for chunk in img_res.iter_content(chunk_size=8192):
                        file.write(chunk)
                
                print(f'{idx}. {file_path} (다운로드 완료)')

            except requests.exceptions.RequestException as e:
                print(f"오류: 이미지 다운로드 실패 ({full_img_url}) - {e}")
            except Exception as e:
                print(f"오류: 파일 저장 중 문제 발생 ({file_path}) - {e}")
        
        print(f"--- '{title}' {no}회차 다운로드 완료 ---")
    else:
        print(f"오류: 페이지 로드 실패 (상태 코드: {res.status_code})")

download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')



--- 웹툰 다운로드 시작 ---
웹툰 제목: 일렉시드, 회차 번호: 341, URL: https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed
총 88개의 이미지를 찾았습니다.
이미지 저장 디렉토리: img\일렉시드\341
디렉토리 'img\일렉시드\341' 생성 완료.
1. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_1.jpg (다운로드 완료)
2. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_2.jpg (다운로드 완료)
3. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_3.jpg (다운로드 완료)
4. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_4.jpg (다운로드 완료)
5. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_5.jpg (다운로드 완료)
6. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_6.jpg (다운로드 완료)
7. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_7.jpg (다운로드 완료)
8. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_8.jpg (다운로드 완료)
9. img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_9.jpg (다운로드 완료)
10. img\일렉시드\341\20250311